In [2]:
import pandas as pd
import numpy as np
import os

In [9]:
list_file = 'experiments_list.txt'
with open(list_file, 'r') as f:
    dirs = [f'experiments/{line.strip()}' for line in f if line.strip() and not line.strip().startswith('#')]

In [25]:
# Load logs.csv from each directory into a dictionary of DataFrames
dfs = {}
for d in dirs:
    csv_path = os.path.join(d, 'logs.csv')
    if os.path.isfile(csv_path):
        dfs[d] = pd.read_csv(csv_path)
    else:
        print(f'Warning: {csv_path} not found, skipping.')

# Optionally, concatenate all DataFrames into a single DataFrame
combined_df = pd.concat(
    dfs.values(), 
    keys=dfs.keys(), 
    names=['experiment', 'row']
).reset_index(level='row', drop=True).reset_index().rename(columns={'index': 'experiment'})

# Display the combined DataFrame
df = combined_df.query('itr>0')[['experiment','iteration_time']].copy().groupby('experiment').mean().reset_index()

In [26]:
df

,experiment,iteration_time
0,experiments/OU_anisotropic_d20/COMBINED/SOCM,0.738791
1,experiments/OU_anisotropic_d20/COMBINED/SOCM_a...,0.569857
2,experiments/OU_anisotropic_d20/COMBINED/log_va...,0.568906
3,experiments/OU_anisotropic_d20/COMBINED/rel_en...,0.613510
4,experiments/OU_anisotropic_d20/EIGF/pinn_GAUSS,0.585661
5,experiments/OU_anisotropic_d20/EIGF/rel_GAUSS,0.786410
6,experiments/OU_anisotropic_d20/EIGF/ritz_GAUSS,0.363406
7,experiments/OU_anisotropic_d20/EIGF/var_GAUSS,0.363305
8,experiments/OU_anisotropic_d20/IDO/SOCM,1.152043
9,experiments/OU_anisotropic_d20/IDO/SOCM_adjoint,0.248138


In [ ]:
parts = df.experiment.str.split('/', expand=True)
df['experiment_name'] = parts[1]
df['method_name'] = parts[2]
df['loss'] = parts[3]

In [45]:
pivot = df.pivot_table(
    index=['method_name', 'loss'],
    columns='experiment_name',
    values='iteration_time',
    aggfunc='mean'       # or 'first', or any other aggregator
)

In [ ]:
pivot['loss'].rename(
    {'SOCM_adjoint': 'SOCM (adjoint)',
     'log_variance': 'Log variance',
     'rel_entropy': 'Relative entropy',
     'pinn_GAUSS': 'PINN',
     'rel_GAUSS': 'Relative loss',
     'ritz_GAUSS': 'Deep ritz loss',
     'var_GAUSS': 'Variational loss'},
     inplace=True
)

pivot['experiment_name'].rename(
    {'OU_anisotropic_d20': 'Stable OU (ANISO)',
     'OU_stable_d20': 'Stable OU (ISO)',
     'OU_hard_d20': 'Unstable OU',
     'double_well_d10': 'Double Well'},
     inplace=True
)

In [51]:
latex_str = pivot.to_latex(
    caption='Iteration times by experiment, method and loss',
    label='tab:iteration_times',
    escape=True,            # <-- ensure special chars (_ & %) get backslash-escaped
    multicolumn=True,
    multicolumn_format='c',
    float_format="%.3f",
    bold_rows=True
)
print(latex_str)

\begin{table}
\caption{Iteration times by experiment, method and loss}
\label{tab:iteration_times}
\begin{tabular}{llrrrr}
\toprule
 & experiment_name & OU\_anisotropic\_d20 & OU\_hard\_d20 & OU\_stable\_d20 & double\_well\_d10 \\
method_name & loss &  &  &  &  \\
\midrule
\multirow[t]{4}{*}{\textbf{COMBINED}} & \textbf{SOCM} & 0.739 & 1.197 & 0.604 & 1.110 \\
\textbf{} & \textbf{SOCM\_adjoint} & 0.570 & 0.808 & 0.466 & 0.739 \\
\textbf{} & \textbf{log\_variance} & 0.569 & 0.806 & 0.459 & 0.745 \\
\textbf{} & \textbf{rel\_entropy} & 0.614 & 0.920 & 5.874 & 0.816 \\
\cline{1-6}
\multirow[t]{4}{*}{\textbf{EIGF}} & \textbf{pinn\_GAUSS} & 0.586 & 0.406 & 0.403 & 1.806 \\
\textbf{} & \textbf{rel\_GAUSS} & 0.786 & 0.405 & 0.404 & 1.811 \\
\textbf{} & \textbf{ritz\_GAUSS} & 0.363 & 0.195 & 0.194 & 1.287 \\
\textbf{} & \textbf{var\_GAUSS} & 0.363 & 0.198 & 0.198 & 1.276 \\
\cline{1-6}
\multirow[t]{4}{*}{\textbf{IDO}} & \textbf{SOCM} & 1.152 & 4.790 & 0.807 & 1.365 \\
\textbf{} & \textbf{SOCM\_

In [52]:
print(latex_str)

\begin{table}
\caption{Iteration times by experiment, method and loss}
\label{tab:iteration_times}
\begin{tabular}{llrrrr}
\toprule
 & experiment_name & OU\_anisotropic\_d20 & OU\_hard\_d20 & OU\_stable\_d20 & double\_well\_d10 \\
method_name & loss &  &  &  &  \\
\midrule
\multirow[t]{4}{*}{\textbf{COMBINED}} & \textbf{SOCM} & 0.739 & 1.197 & 0.604 & 1.110 \\
\textbf{} & \textbf{SOCM\_adjoint} & 0.570 & 0.808 & 0.466 & 0.739 \\
\textbf{} & \textbf{log\_variance} & 0.569 & 0.806 & 0.459 & 0.745 \\
\textbf{} & \textbf{rel\_entropy} & 0.614 & 0.920 & 5.874 & 0.816 \\
\cline{1-6}
\multirow[t]{4}{*}{\textbf{EIGF}} & \textbf{pinn\_GAUSS} & 0.586 & 0.406 & 0.403 & 1.806 \\
\textbf{} & \textbf{rel\_GAUSS} & 0.786 & 0.405 & 0.404 & 1.811 \\
\textbf{} & \textbf{ritz\_GAUSS} & 0.363 & 0.195 & 0.194 & 1.287 \\
\textbf{} & \textbf{var\_GAUSS} & 0.363 & 0.198 & 0.198 & 1.276 \\
\cline{1-6}
\multirow[t]{4}{*}{\textbf{IDO}} & \textbf{SOCM} & 1.152 & 4.790 & 0.807 & 1.365 \\
\textbf{} & \textbf{SOCM\_